# HW3 Recurent Neural Network

## Overview

In this homework, you will build a bi-directional RNN on diagnosis codes. The recurrent nature of RNN allows us to model the temporal relation of different visits of a patient. More specifically, we will still perform **Heart Failure Prediction**, but with different input formats.

---

In [1]:
import os
import sys
import pickle
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# set seed
seed = 24
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)


# Define data path
DATA_PATH = "../HW3_RNN-lib/data"

---

## About Raw Data

To get started, we will implement a naive RNN model for heart failure prediction using the diagnosis codes.

We will use the same dataset synthesized from [MIMIC-III](https://mimic.physionet.org/gettingstarted/access/), but with different input formats.

The data has been preprocessed for you. Let us load them and take a look.

In [2]:
pids = pickle.load(open(os.path.join(DATA_PATH,'train/pids.pkl'), 'rb'))
vids = pickle.load(open(os.path.join(DATA_PATH,'train/vids.pkl'), 'rb'))
hfs = pickle.load(open(os.path.join(DATA_PATH,'train/hfs.pkl'), 'rb'))
seqs = pickle.load(open(os.path.join(DATA_PATH,'train/seqs.pkl'), 'rb'))
types = pickle.load(open(os.path.join(DATA_PATH,'train/types.pkl'), 'rb'))
rtypes = pickle.load(open(os.path.join(DATA_PATH,'train/rtypes.pkl'), 'rb'))

assert len(pids) == len(vids) == len(hfs) == len(seqs) == 1000
assert len(types) == 619

where

- `pids`: contains the patient ids
- `vids`: contains a list of visit ids for each patient
- `hfs`: contains the heart failure label (0: normal, 1: heart failure) for each patient
- `seqs`: contains a list of visit (in ICD9 codes) for each patient
- `types`: contains the map from ICD9 codes to ICD-9 labels
- `rtypes`: contains the map from ICD9 labels to ICD9 codes

Let us take a patient as an example.

In [3]:
# take the 3rd patient as an example

print("Patient ID:", pids[3])
print("Heart Failure:", hfs[3])
print("# of visits:", len(vids[3]))
for visit in range(len(vids[3])):
    print(f"\t{visit}-th visit id:", vids[3][visit])
    print(f"\t{visit}-th visit diagnosis labels:", seqs[3][visit])
    print(f"\t{visit}-th visit diagnosis codes:", [rtypes[label] for label in seqs[3][visit]])

Patient ID: 47537
Heart Failure: 0
# of visits: 2
	0-th visit id: 0
	0-th visit diagnosis labels: [12, 103, 262, 285, 290, 292, 359, 416, 39, 225, 275, 294, 326, 267, 93]
	0-th visit diagnosis codes: ['DIAG_041', 'DIAG_276', 'DIAG_518', 'DIAG_560', 'DIAG_567', 'DIAG_569', 'DIAG_707', 'DIAG_785', 'DIAG_155', 'DIAG_456', 'DIAG_537', 'DIAG_571', 'DIAG_608', 'DIAG_529', 'DIAG_263']
	1-th visit id: 1
	1-th visit diagnosis labels: [12, 103, 240, 262, 290, 292, 319, 359, 510, 513, 577, 307, 8, 280, 18, 131]
	1-th visit diagnosis codes: ['DIAG_041', 'DIAG_276', 'DIAG_482', 'DIAG_518', 'DIAG_567', 'DIAG_569', 'DIAG_599', 'DIAG_707', 'DIAG_995', 'DIAG_998', 'DIAG_V09', 'DIAG_584', 'DIAG_031', 'DIAG_553', 'DIAG_070', 'DIAG_305']


pids → list/array of patient IDs

hfs → list/array that says if a patient has heart failure (True/False or 1/0)

vids → list of lists → each patient has multiple visit IDs

seqs → list of lists → each patient has multiple visits, and each visit has diagnosis labels (numbers)

rtypes → dictionary that maps diagnosis labels (numbers) to their actual diagnosis codes/names

Loop through each visit for this patient:

vids[3][visit] = the visit’s ID

seqs[3][visit] = the list of diagnosis labels for that visit (like [12, 45, 88])

[rtypes[label] for label in seqs[3][visit]] = convert those labels into actual diagnosis codes (e.g., ["I50.9", "E11.9", "J45.9"])

Note that `seqs` is a list of list of list. That is, `seqs[i][j][k]` gives you the k-th diagnosis codes for the j-th visit for the i-th patient.

And you can look up the meaning of the ICD9 code online. For example, `DIAG_276` represetns *disorders of fluid electrolyte and acid-base balance*.

Further, let see number of heart failure patients.

In [4]:
print("number of heart failure patients:", sum(hfs))
print("ratio of heart failure patients: %.2f" % (sum(hfs) / len(hfs)))

number of heart failure patients: 548
ratio of heart failure patients: 0.55


Now we have the data. Let us build the naive RNN.

## 1 Build the dataset [30 points]

### 1.1 CustomDataset [5 points]

First, let us implement a custom dataset using PyTorch class `Dataset`, which will characterize the key features of the dataset we want to generate.

We will use the sequences of diagnosis codes `seqs` as input and heart failure `hfs` as output.

In [5]:
from torch.utils.data import Dataset


class CustomDataset(Dataset):
    
    def __init__(self, seqs, hfs):
        
        """
        TODO: Store `seqs`. to `self.x` and `hfs` to `self.y`.
        
        Note that you DO NOT need to covert them to tensor as we will do this later.
        Do NOT permute the data.
        """
        
        # your code here
        # Just store references
        self.x = seqs
        self.y = hfs
        #raise NotImplementedError
         # (Optional) sanity check
        if len(self.x) != len(self.y):
            raise ValueError(f"Mismatched lengths: len(seqs)={len(self.x)} != len(hfs)={len(self.y)}")

    
    def __len__(self):
        
        """
        TODO: Return the number of samples (i.e. patients).
        """
        
        # your code here
        return len(self.x)
        #raise NotImplementedError
    
    def __getitem__(self, index):
        
        """
        TODO: Generates one sample of data.
        
        Note that you DO NOT need to covert them to tensor as we will do this later.
        """
        
        # your code here
        return 
        #raise NotImplementedError
        self.x[index], self.y[index]
        

dataset = CustomDataset(seqs, hfs)

self.x = [[visit1, visit2], [visit1]]   # 2 patients
self.y = [0, 1, 1]                      # 3 labels

Here, inputs = 2 patients, labels = 3 → mismatch ⚠️


In [6]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, seqs, hfs):
        """
        Store the sequences and labels/targets without any permutation
        and WITHOUT converting to tensors (conversion can be done later
        in a collate_fn or inside the training loop).

        Parameters
        ----------
        seqs : Sequence
            Input sequences/features (e.g., list of patient sequences).
        hfs : Sequence
            Targets/labels corresponding 1‑to‑1 with seqs.
        """
        # Just store references
        self.x = seqs
        self.y = hfs

        # (Optional) sanity check
        if len(self.x) != len(self.y):
            raise ValueError(f"Mismatched lengths: len(seqs)={len(self.x)} != len(hfs)={len(self.y)}")

    def __len__(self):
        """
        Return the number of samples (patients).
        """
        return len(self.x)

    def __getitem__(self, index):
        """
        Retrieve a single sample by index.

        Returns
        -------
        tuple
            (sequence, target) pair for the given index.
        """
        return self.x[index], self.y[index]

In [7]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''

dataset = CustomDataset(seqs, hfs)

assert len(dataset) == 1000



### 1.2 Collate Function [20 points]

As you note that, we do not convert the data to tensor in the built `CustomDataset`. Instead, we will do this using a collate function `collate_fn()`. 

This collate function `collate_fn()` will be called by `DataLoader` after fetching a list of samples using the indices from `CustomDataset` to collate the list of samples into batches.

For example, assume the `DataLoader` gets a list of two samples.

```
[ [ [0, 1, 2], [8, 0] ], 
  [ [12, 13, 6, 7], [12], [23, 11] ] ]
```

where the first sample has two visits `[0, 1, 2]` and `[8, 0]` and the second sample has three visits `[12, 13, 6, 7]`, `[12]`, and `[23, 11]`.

The collate function `collate_fn()` is supposed to pad them into the same shape (3, 4), where 3 is the maximum number of visits and 4 is the maximum number of diagnosis codes.

``` 
[ [ [0, 1, 2, *0*], [8, 0, *0*, *0*], [*0*, *0*, *0*, *0*]  ], 
  [ [12, 13, 6, 7], [12, *0*, *0*, *0*], [23, 11, *0*, *0*] ] ]
```

Further, the padding information will be stored in a mask with the same shape, where 1 indicates that the diagnosis code at this position is from the original input, and 0 indicates that the diagnosis code at this position is the padded value.

```
[ [ [1, 1, 1, 0], [1, 1, 0, 0], [0, 0, 0, 0] ], 
  [ [1, 1, 1, 1], [1, 0, 0, 0], [1, 1, 0, 0] ] ]
```

Lastly, we will have another diagnosis sequence in reversed time. This will be used in our RNN model for masking. Note that we only flip the true visits.

``` 
[ [ [8, 0, *0*, *0*], [0, 1, 2, *0*], [*0*, *0*, *0*, *0*]  ], 
  [ [23, 11, *0*, *0*], [12, *0*, *0*, *0*], [12, 13, 6, 7] ] ]
```

And a reversed mask as well.

```
[ [ [1, 1, 0, 0], [1, 1, 1, 0], [0, 0, 0, 0] ], 
  [ [1, 1, 0, 0], [1, 0, 0, 0], [1, 1, 1, 1], ] ]
```

We need to pad the sequences into the same length so that we can do batch training on GPU. And we also need this mask so that when training, we can ignored the padded value as they actually do not contain any information.

Great! Let’s visualize it step by step with a simple diagram.

---

### 🎯 Example: 2 patients

* Patient 1 has 2 visits
* Patient 2 has 3 visits

#### Raw data

```
Patient 1: [ [0, 1, 2], [8, 0] ]
Patient 2: [ [12, 13, 6, 7], [12], [23, 11] ]
```

---

### Step 1: Pad to same shape (3 visits × 4 codes)

```
Patient 1: [ [0,  1,  2,  0],
             [8,  0,  0,  0],
             [0,  0,  0,  0] ]

Patient 2: [ [12, 13, 6,  7],
             [12,  0, 0,  0],
             [23, 11, 0,  0] ]
```

---

### Step 2: Mask (1 = real value, 0 = padding)

```
Patient 1 mask: [ [1, 1, 1, 0],
                  [1, 1, 0, 0],
                  [0, 0, 0, 0] ]

Patient 2 mask: [ [1, 1, 1, 1],
                  [1, 0, 0, 0],
                  [1, 1, 0, 0] ]
```

---

### Step 3: Reverse visits (most recent first)

```
Patient 1 reversed: [ [8,  0, 0, 0],
                      [0,  1, 2, 0],
                      [0,  0, 0, 0] ]

Patient 2 reversed: [ [23, 11, 0, 0],
                      [12,  0, 0, 0],
                      [12, 13, 6, 7] ]
```

---

### Step 4: Reverse mask

```
Patient 1 reversed mask: [ [1, 1, 0, 0],
                           [1, 1, 1, 0],
                           [0, 0, 0, 0] ]

Patient 2 reversed mask: [ [1, 1, 0, 0],
                           [1, 0, 0, 0],
                           [1, 1, 1, 1] ]
```

---

✨ **Big picture diagram**

```
Raw data  →  Padded data  →  Mask
                    ↓
             Reversed data → Reversed mask
```

---

✅ Now, the model can process everything in neat, same-sized batches — and thanks to the mask, it knows which numbers are “real” and which are just padding.

---




In [8]:
import torch

def collate_fn(data):
    """
    Collate a list of samples (patient visit sequences) into padded batch tensors.

    Each element of `data` is a tuple: (patient_sequence, label)
        patient_sequence: list[ list[int] ]  # visits -> diagnosis code ids
        label: float or int (heart failure label)

    Returns
    -------
    x : LongTensor   (B, V_max, C_max)
        Padded diagnosis code ids in chronological order.
    masks : BoolTensor (B, V_max, C_max)
        True where a real code exists, False for padding.
    rev_x : LongTensor (B, V_max, C_max)
        Same codes but visits reversed in time (compressed to the front, then padded).
    rev_masks : BoolTensor (B, V_max, C_max)
        Mask for rev_x.
    y : FloatTensor (B,)
        Labels.
    """
    sequences, labels = zip(*data)          # sequences: list of patients; labels: list
    y = torch.tensor(labels, dtype=torch.float)

    num_patients = len(sequences)
    num_visits_per_patient = [len(p) for p in sequences]
    num_codes_all_visits = [len(v) for p in sequences for v in p]

    max_num_visits = max(num_visits_per_patient) if num_visits_per_patient else 0
    max_num_codes = max(num_codes_all_visits) if num_codes_all_visits else 0

    x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    rev_x = torch.zeros_like(x)
    masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
    rev_masks = torch.zeros_like(masks)

    for i_patient, patient in enumerate(sequences):
        num_v = len(patient)
        for j_visit, visit in enumerate(patient):
            # Convert visit (list of code ids) to tensor
            codes = torch.as_tensor(visit, dtype=torch.long)

            # Chronological placement
            code_len = len(codes)
            x[i_patient, j_visit, :code_len] = codes
            masks[i_patient, j_visit, :code_len] = True

            # Reversed placement (compress visits to the front in reversed temporal order)
            rev_j = num_v - 1 - j_visit
            rev_x[i_patient, rev_j, :code_len] = codes
            rev_masks[i_patient, rev_j, :code_len] = True

    return x, masks, rev_x, rev_masks, y

Perfect, let’s **walk through this code with a toy dataset** so you can *see exactly* how `collate_fn` transforms the data.

---

### 🔹 Step 1: Tiny dataset

Let’s say we have **2 patients** (batch size = 2). Each patient has a different number of visits, and each visit has a different number of diagnosis codes.

```python
data = [
    ( [[1, 2], [3]], 0 ),             # Patient 1 → 2 visits → label = 0
    ( [[4, 5, 6], [7], [8, 9]], 1 )   # Patient 2 → 3 visits → label = 1
]
```

* Patient 1:

  * Visit 1: codes \[1, 2]
  * Visit 2: codes \[3]
  * Label: `0`
* Patient 2:

  * Visit 1: codes \[4, 5, 6]
  * Visit 2: codes \[7]
  * Visit 3: codes \[8, 9]
  * Label: `1`

---

### 🔹 Step 2: Figure out maximum sizes

* Max number of visits = **3** (Patient 2 has 3 visits)
* Max number of codes in any visit = **3** (visit \[4,5,6])

So the padded tensors will have shape:

```
(Batch, MaxVisits, MaxCodes) = (2, 3, 3)
```

---

### 🔹 Step 3: Chronological padded tensor `x`

Fill each patient’s visits in order, pad with `0`s.

```
Patient 1 (2 visits, pad to 3 visits):
[[1, 2, 0],    # visit 1
 [3, 0, 0],    # visit 2
 [0, 0, 0]]    # padded visit

Patient 2 (3 visits, no padding needed):
[[4, 5, 6],    # visit 1
 [7, 0, 0],    # visit 2
 [8, 9, 0]]    # visit 3
```

So:

```
x =
[
  [[1, 2, 0], [3, 0, 0], [0, 0, 0]],
  [[4, 5, 6], [7, 0, 0], [8, 9, 0]]
]
```

---

### 🔹 Step 4: Chronological mask `masks`

Same shape as `x`, with `True` for real codes, `False` for padding.

```
Patient 1:
[[1, 1, 0],   # [1,2]
 [1, 0, 0],   # [3]
 [0, 0, 0]]   # padding

Patient 2:
[[1, 1, 1],   # [4,5,6]
 [1, 0, 0],   # [7]
 [1, 1, 0]]   # [8,9]
```

---

### 🔹 Step 5: Reversed visits `rev_x`

Visits are flipped in **time order** (most recent first).

```
Patient 1 reversed (2 visits):
[[3, 0, 0],   # visit 2
 [1, 2, 0],   # visit 1
 [0, 0, 0]]   # padding

Patient 2 reversed (3 visits):
[[8, 9, 0],   # visit 3
 [7, 0, 0],   # visit 2
 [4, 5, 6]]   # visit 1
```

---

### 🔹 Step 6: Reversed mask `rev_masks`

```
Patient 1:
[[1, 0, 0],
 [1, 1, 0],
 [0, 0, 0]]

Patient 2:
[[1, 1, 0],
 [1, 0, 0],
 [1, 1, 1]]
```

---

### 🔹 Step 7: Labels `y`

Simply:

```
y = [0, 1]
```

---

### ✅ Final Output of `collate_fn`

1. `x` → padded chronological sequences
2. `masks` → mask for `x`
3. `rev_x` → padded reversed sequences
4. `rev_masks` → mask for `rev_x`
5. `y` → labels

---

📊 **Visualization Summary**

```
Input data (ragged):
Patient 1: [[1,2], [3]]                  Label=0
Patient 2: [[4,5,6], [7], [8,9]]         Label=1

↓ Collate_fn pads & masks ↓

x = [[[1,2,0], [3,0,0], [0,0,0]],
     [[4,5,6], [7,0,0], [8,9,0]]]

masks = [[[1,1,0], [1,0,0], [0,0,0]],
         [[1,1,1], [1,0,0], [1,1,0]]]

rev_x = [[[3,0,0], [1,2,0], [0,0,0]],
         [[8,9,0], [7,0,0], [4,5,6]]]

rev_masks = [[[1,0,0], [1,1,0], [0,0,0]],
             [[1,1,0], [1,0,0], [1,1,1]]]

y = [0, 1]
```

---

Would you like me to also **draw this as a block diagram** (boxes for patients → padded batch → reversed batch) so you can *see* the transformation flow?


In [9]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''

from torch.utils.data import DataLoader

loader = DataLoader(dataset, batch_size=10, collate_fn=collate_fn)
loader_iter = iter(loader)
x, masks, rev_x, rev_masks, y = next(loader_iter)

assert x.dtype == rev_x.dtype == torch.long
assert y.dtype == torch.float
assert masks.dtype == rev_masks.dtype == torch.bool

assert x.shape == rev_x.shape == masks.shape == rev_masks.shape == (10, 3, 24)
assert y.shape == (10,)



Now we have `CustomDataset` and `collate_fn()`. Let us split the dataset into training and validation sets.

In [10]:
from torch.utils.data.dataset import random_split

split = int(len(dataset)*0.8)

lengths = [split, len(dataset) - split]
train_dataset, val_dataset = random_split(dataset, lengths)

print("Length of train dataset:", len(train_dataset))
print("Length of val dataset:", len(val_dataset))

Length of train dataset: 800
Length of val dataset: 200


### 1.3 DataLoader [5 points]

Now, we can load the dataset into the data loader.

In [11]:
from torch.utils.data import DataLoader

def load_data(train_dataset, val_dataset, collate_fn):
    
    '''
    TODO: Implement this function to return the data loader for  train and validation dataset. 
    Set batchsize to 32. Set `shuffle=True` only for train dataloader.
    
    Arguments:
        train dataset: train dataset of type `CustomDataset`
        val dataset: validation dataset of type `CustomDataset`
        collate_fn: collate function
        
    Outputs:
        train_loader, val_loader: train and validation dataloaders
    
    Note that you need to pass the collate function to the data loader `collate_fn()`.
    '''
    
    batch_size = 32
    # your code here
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,          # Shuffle only for training
        collate_fn=collate_fn,
        drop_last=False,
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,         # No shuffle for validation
        collate_fn=collate_fn,
        drop_last=False,
    )
    #raise NotImplementedError
    
    return train_loader, val_loader


train_loader, val_loader = load_data(train_dataset, val_dataset, collate_fn)

In [12]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''

train_loader, val_loader = load_data(train_dataset, val_dataset, collate_fn)

assert len(train_loader) == 25, "Length of train_loader should be 25, instead we got %d"%(len(train_loader))



## 2 Naive RNN [35 points] 

Let us implement a naive bi-directional RNN model.

<img src="img/bi-rnn.jpg" width="600"/>

Remember from class that, first of all, we need to transform the diagnosis code for each visit of a patient to an embedding. To do this, we can use `nn.Embedding()`, where `num_embeddings` is the number of diagnosis codes and `embedding_dim` is the embedding dimension.

Then, we can construct a simple RNN structure. Each input is this multi-hot vector. At the 0-th visit, this has $\boldsymbol{X}_0$, and at t-th visit, this has $\boldsymbol{X}_t$.

Each one of the input will then map to a hidden state $\boldsymbol{\overleftrightarrow{h}}_t$. The forward hidden state $\boldsymbol{\overrightarrow{h}}_t$ can be determined by $\boldsymbol{\overrightarrow{h}}_{t-1}$ and the corresponding current input $\boldsymbol{X}_t$.

Similarly, we will have another RNN to process the sequence in the reverse order, so that the hidden state $\boldsymbol{\overleftarrow{h}}_t$ is determined by $\boldsymbol{\overleftarrow{h}}_{t+1}$ and $\boldsymbol{X}_t$.

Finally, once we have the $\boldsymbol{\overrightarrow{h}}_T$ and $\boldsymbol{\overleftarrow{h}}_{0}$, we will concatenate the two vectors as the feature vector and train a NN to perform the classification.

Now, let us build this model. The forward steps will be:

    1. Pass the sequence through the embedding layer;
    2. Sum the embeddings for each diagnosis code up for a visit of a patient;
    3. Pass the embeddings through the RNN layer;
    4. Obtain the hidden state at the last visit;
    5. Do 1-4 for both directions and concatenate the hidden states.
    6. Pass the hidden state through the linear and activation layers.

Simple explanation:
Main Ideas

Turn codes into vectors (Embeddings):

Each diagnosis code is just a number (like 42 for diabetes).

The nn.Embedding() layer converts these numbers into useful vectors (embeddings) so the model can understand them better.

Visits = sets of codes:

A patient’s visit has multiple diagnosis codes.

We sum all the embeddings for the visit → one vector per visit.

Process visits in order (RNN):

The RNN takes visits one by one (like reading a sentence word by word).

At each visit, it keeps track of the current state + previous state.

This gives us the forward hidden state (h→t).

Process visits in reverse (Bi-RNN):

We also process the visits backwards (from last to first).

This gives us the backward hidden state (h←t).

Combine both directions:

At the end, we take the last forward state and the last backward state.

Concatenate (stick them together) → one big vector with information from past and future visits.

Final classification:

Pass this combined vector through a linear layer (like a final decision layer).

Use an activation (like sigmoid) to predict if the patient has heart failure (1) or not (0).

🧠 Think of it like:

Embedding layer = dictionary that translates codes into meaningful words.

Forward RNN = reading the patient’s history forward.

Backward RNN = reading the history backward.

Concatenation = combining both perspectives.

Final layer = doctor makes a yes/no decision.

👉 So in short:
We embed diagnosis codes → summarize visits → run them through RNN forward + backward → combine → classify.

Great question 🙌 Let’s clear that up.

In this **naive Bi-RNN setup**, we **don’t use all visits’ hidden states**. Instead:

* We process all visits forward → at the **last visit** we get the **last forward hidden state** (`h→T`).
* We process all visits backward → at the **first visit** we get the **last backward hidden state** (`h←0`).

👉 Then we **only take these two final states** (not all visits), concatenate them:

```
h_final = [ h→T ; h←0 ]- thhis is the summary of all visit
```

and pass that through the **Linear layer** for classification.

---

🔑 Why?
Because this model just wants a **summary of the whole patient history** (forward & backward), not predictions at each visit.

---

✅ But in more advanced models (like sequence-to-sequence, attention, or Transformers), yes—you often combine **all hidden states from every visit** before classification.

---



### 2.1 Mask Selection [20 points]

Importantly, you need to use `masks` to mask out the paddings in before step 2 and before 4. So, let us first preform the mask selection.

In [13]:
def sum_embeddings_with_mask(x, masks):
    """
    TODO: mask select the embeddings for true visits (not padding visits) and then
        sum the embeddings for each visit up.

    Arguments:
        x: the embeddings of diagnosis sequence of shape (batch_size, # visits, # diagnosis codes, embedding_dim)
        masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

    Outputs:
        sum_embeddings: the sum of embeddings of shape (batch_size, # visits, embedding_dim)
        
    NOTE: Do NOT use for loop.

    """
    
    # your code here
     # Ensure boolean -> float for multiplication (broadcast over embedding_dim)
    mask_float = masks.unsqueeze(-1).to(dtype=x.dtype)          # (B, V, C, 1)

    # Zero out padded positions then sum across code dimension
    sum_embeddings = (x * mask_float).sum(dim=2)
    
    return sum_embeddings

    #raise NotImplementedError

how this function work with real data
Perfect 👍 Let’s **walk through this with a real mini dataset** so you can see how `sum_embeddings_with_mask()` works step by step.

---

### 📌 Recall what the function does

* `x`: embeddings for each diagnosis code → shape = `(B, V, C, D)`

  * `B = batch size (patients)`
  * `V = number of visits`
  * `C = number of codes per visit`
  * `D = embedding dimension`
* `masks`: tells us which entries are **real codes (1)** vs \*\*padding (0)`. Shape = `(B, V, C)\`.
* Output: sum embeddings per visit → `(B, V, D)`.

---

### 🧪 Example data

Let’s say:

* Batch size = 1 patient (`B=1`)
* Visits = 2 (`V=2`)
* Codes per visit = 3 (`C=3`)
* Embedding dimension = 2 (`D=2`)

```python
import torch

# Embeddings (B=1, V=2, C=3, D=2)
x = torch.tensor([
    [   # Patient 1
        [[1., 1.], [2., 2.], [0., 0.]],   # Visit 1: two real codes, one padding
        [[3., 3.], [4., 4.], [5., 5.]]    # Visit 2: three real codes
    ]
])

# Masks (B=1, V=2, C=3)
masks = torch.tensor([
    [
        [1, 1, 0],   # Visit 1: first two codes real, last one padding
        [1, 1, 1]    # Visit 2: all three codes real
    ]
])

def sum_embeddings_with_mask(x, masks):
    mask_float = masks.unsqueeze(-1).to(dtype=x.dtype)   # (B, V, C, 1)
    sum_embeddings = (x * mask_float).sum(dim=2)        # sum over codes
    return sum_embeddings

sum_embeddings_with_mask(x, masks)
```

---

### ✅ What happens?

1. `x` (embeddings):

   ```
   Visit 1: [[1,1], [2,2], [0,0]]  
   Visit 2: [[3,3], [4,4], [5,5]]  
   ```

2. `masks`:

   ```
   Visit 1: [1, 1, 0]  (last code ignored)  
   Visit 2: [1, 1, 1]  (all used)  
   ```

3. Multiply `x * masks`:

   ```
   Visit 1: [[1,1], [2,2], [0,0]]  
   Visit 2: [[3,3], [4,4], [5,5]]  
   ```

   (nothing changes for real codes, padding stays zero)

4. Sum across codes (`dim=2`):

   ```
   Visit 1 sum: [3,3]   ( [1,1] + [2,2] )  
   Visit 2 sum: [12,12] ( [3,3] + [4,4] + [5,5] )  
   ```

---

### 🎯 Final Output

Shape = `(B=1, V=2, D=2)`

```
tensor([[
    [ 3.,  3.],   # Visit 1
    [12., 12.]    # Visit 2
]])
```

---

👉 So the function **zeroes out padded codes** and **sums only the real code embeddings per visit**.

---

Would you like me to also show this with a **batch of 2 patients**, so you see how it works across multiple patients at once?


In [14]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''

import random
import ast
import inspect


def uses_loop(function):
    loop_statements = ast.For, ast.While, ast.AsyncFor

    nodes = ast.walk(ast.parse(inspect.getsource(function)))
    return any(isinstance(node, loop_statements) for node in nodes)

def generate_random_mask(batch_size, max_num_visits , max_num_codes):
    num_visits = [random.randint(1, max_num_visits) for _ in range(batch_size)]
    num_codes = []
    for n in num_visits:
        num_codes_visit = [0] * max_num_visits
        for i in range(n):
            num_codes_visit[i] = (random.randint(1, max_num_codes))
        num_codes.append(num_codes_visit)
    masks = [torch.ones((l,), dtype=torch.bool) for num_codes_visit in num_codes for l in num_codes_visit]
    masks = torch.stack([torch.cat([i, i.new_zeros(max_num_codes - i.size(0))], 0) for i in masks], 0)
    masks = masks.view((batch_size, max_num_visits, max_num_codes)).bool()
    return masks


batch_size = 16
max_num_visits = 10
max_num_codes = 20
embedding_dim = 100

torch.random.manual_seed(7)
x = torch.randn((batch_size, max_num_visits , max_num_codes, embedding_dim))
masks = generate_random_mask(batch_size, max_num_visits , max_num_codes)
out = sum_embeddings_with_mask(x, masks)

assert uses_loop(sum_embeddings_with_mask) is False
assert out.shape == (batch_size, max_num_visits, embedding_dim)




In [15]:
import torch

def get_last_visit(hidden_states, masks):
    """
    Obtain the hidden state for the LAST real (non‑padding) visit for each patient.

    Parameters
    ----------
    hidden_states : torch.Tensor
        Shape (B, V, E) where
          B = batch size
          V = max # visits (padded)
          E = embedding dim
        Each row along V corresponds to a visit's hidden state (already aggregated per visit).
    masks : torch.Tensor (bool)
        Shape (B, V, C) visit-level padding mask over diagnosis codes.
        A visit is considered 'real' if ANY code in that visit is True.

    Returns
    -------
    last_hidden_state : torch.Tensor
        Shape (B, E) containing the hidden state of the last real visit for every patient.

    Notes
    -----
    - No Python for-loops are used.
    - Uses torch.gather as hinted.
    - Assumes that padded visits (all-False) appear only after real visits (standard padding).
      If there are interior empty visits, the function still picks the last visit that has at least
      one True code.
    """
    # 1. Derive a (B, V) boolean mask indicating which visits are real
    visit_mask = masks.any(dim=2)                   # (B, V)

    # 2. Build an index tensor of visit positions 0..V-1
    B, V, E = hidden_states.shape
    positions = torch.arange(V, device=hidden_states.device).unsqueeze(0)  # (1, V)

    # 3. Zero out positions where visit is padding (or set to -1) to ignore them.
    # Using -1 ensures they won't become the max unless all visits are padding.
    masked_positions = positions.masked_fill(~visit_mask, -1)

    # 4. Find the index of the last real visit (largest position that is real)
    last_indices = masked_positions.argmax(dim=1)   # (B,)

    # (Optional) If you want to guard against patients with no real visits, you can add:
    # if (visit_mask.sum(dim=1) == 0).any():
    #     raise ValueError("At least one patient has no real (non-padding) visits.")

    # 5. Gather the hidden states at those indices
    gather_index = last_indices.view(B, 1, 1).expand(B, 1, E)  # (B, 1, E)
    last_hidden_state = hidden_states.gather(dim=1, index=gather_index).squeeze(1)  # (B, E)

    return last_hidden_state


In [16]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''

assert uses_loop(get_last_visit) is False

max_num_visits = 10
batch_size = 16
max_num_codes = 20
embedding_dim = 100

torch.random.manual_seed(7)
hidden_states = torch.randn((batch_size, max_num_visits, embedding_dim))
masks = generate_random_mask(batch_size, max_num_visits , max_num_codes)
out = get_last_visit(hidden_states, masks)

assert out.shape == (batch_size, embedding_dim)




Lat visit calculation
Perfect! Let’s go **step by step** with the 4-patient toy data we used earlier. This will show exactly how `get_last_visit()` works.

---

## 1. Recall the data

### Hidden states (summed embeddings per visit) = `hidden_states`

Shape = `(B=4, V=2, E=2)`

```
Patient 1 → [[ 3,  3], [12, 12]]
Patient 2 → [[ 1,  1], [13, 13]]
Patient 3 → [[ 9,  9], [ 0,  0]]
Patient 4 → [[ 5,  5], [ 6,  6]]
```

### Masks (`masks`)

Shape = `(B=4, V=2, C=3)`

```
Patient 1: [[1,1,0], [1,1,1]]
Patient 2: [[1,0,0], [1,1,0]]
Patient 3: [[1,1,1], [0,0,0]]
Patient 4: [[1,0,0], [1,1,1]]
```

---

## 2. Step 1: Determine which visits are real

```python
visit_mask = masks.any(dim=2)   # (B, V)
```

* `True` if **any code in the visit is real**

```
Patient 1: [True, True]      # both visits have codes
Patient 2: [True, True]
Patient 3: [True, False]     # second visit is padding
Patient 4: [True, True]
```

---

## 3. Step 2 & 3: Build positions and mask padding

```python
positions = [[0,1]]   # visit indices along dim=1
masked_positions = positions.masked_fill(~visit_mask, -1)
```

After masking padded visits (False → -1):

```
Patient 1: [0,1]
Patient 2: [0,1]
Patient 3: [0,-1]
Patient 4: [0,1]
```

---

## 4. Step 4: Find last real visit

```python
last_indices = masked_positions.argmax(dim=1)  # largest valid index
```

* Picks **largest position that is real** (last visit)

```
Patient 1: 1  # Visit 2
Patient 2: 1  # Visit 2
Patient 3: 0  # Visit 1
Patient 4: 1  # Visit 2
```

---

## 5. Step 5: Gather hidden states

```python
gather_index = last_indices.view(B,1,1).expand(B,1,E)
last_hidden_state = hidden_states.gather(dim=1, index=gather_index).squeeze(1)
```

* Picks hidden state at **last real visit** for each patient

```
Patient 1: [12, 12]  # Visit 2
Patient 2: [13, 13]  # Visit 2
Patient 3: [9, 9]    # Visit 1 (second is padding)
Patient 4: [6, 6]    # Visit 2
```

---

### ✅ Final Output (`last_hidden_state`)

```
tensor([
 [12, 12],
 [13, 13],
 [ 9,  9],
 [ 6,  6]
])
```

---

### 💡 Summary

1. `masks.any(dim=2)` → figure out which visits have real codes.
2. Build visit **indices**, mask out padded visits with -1.
3. `argmax` → find **last real visit**.
4. `gather` → pick hidden states at those indices.



### 2.2 Build NaiveRNN [15 points]

In [17]:

class NaiveRNN(nn.Module):
    """
    A simple dual-direction (manual bidirectional) RNN model:
      1. Embed diagnosis codes.
      2. Sum code embeddings within each visit (masking padding codes).
      3. Run a forward GRU over visits.
      4. Run a reverse GRU over visits constructed from reversed visit order.
      5. Extract the last real visit hidden state from each direction.
      6. Concatenate forward + reverse states (size 256) -> Linear -> Sigmoid.
    """

    def __init__(self, num_codes):
        super().__init__()
        embDimSize = 128
        hidden_size = 128

        # 1. Embedding layer (assumes 0 is padding index)
        self.embedding = nn.Embedding(
            num_embeddings=num_codes,
            embedding_dim=embDimSize,
            padding_idx=0
        )

        # 2. Forward GRU (visit-level)
        self.rnn = nn.GRU(
            input_size=embDimSize,
            hidden_size=hidden_size,
            batch_first=True
        )

        # 3. Reverse GRU (will process reversed visit order)
        self.rev_rnn = nn.GRU(
            input_size=embDimSize,
            hidden_size=hidden_size,
            batch_first=True
        )

        # 4. Linear head (concat forward + reverse -> 256 -> 1)
        self.fc = nn.Linear(2 * hidden_size, 1)

        # 5. Sigmoid activation
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, masks, rev_x, rev_masks):
        """
        x:        (B, V, C) diagnosis code indices (forward order)
        masks:    (B, V, C) boolean mask for x
        rev_x:    (B, V, C) diagnosis code indices (reversed visits)
        rev_masks:(B, V, C) boolean mask for rev_x

        Returns:
            probs: (B,) probability per patient
        """
        batch_size = x.size(0)

        # ----- Forward direction -----
        # Step 1: embed
        emb_forward = self.embedding(x)              # (B, V, C, E)
        # Step 2: sum code embeddings per visit
        visit_emb_forward = sum_embeddings_with_mask(emb_forward, masks)   # (B, V, E)
        # Step 3: GRU over visits
        output_forward, _ = self.rnn(visit_emb_forward)                     # (B, V, H)
        # Step 4: last real visit hidden state
        true_h_n_forward = get_last_visit(output_forward, masks)            # (B, H)

        # ----- Reverse direction (same steps with reversed visit order) -----
        emb_reverse = self.embedding(rev_x)                                 # (B, V, C, E)
        visit_emb_reverse = sum_embeddings_with_mask(emb_reverse, rev_masks)# (B, V, E)
        output_reverse, _ = self.rev_rnn(visit_emb_reverse)                 # (B, V, H)
        true_h_n_reverse = get_last_visit(output_reverse, rev_masks)        # (B, H)

        # Concatenate forward + reverse final states -> (B, 2H)
        concat_h = torch.cat([true_h_n_forward, true_h_n_reverse], dim=1)    # (B, 256)

        # Linear + sigmoid
        logits = self.fc(concat_h)                                          # (B, 1)
        probs = self.sigmoid(logits).view(batch_size)                       # (B,)
      
        return probs.view(batch_size)
    

# load the model here
naive_rnn = NaiveRNN(num_codes = len(types))
naive_rnn

NaiveRNN(
  (embedding): Embedding(619, 128, padding_idx=0)
  (rnn): GRU(128, 128, batch_first=True)
  (rev_rnn): GRU(128, 128, batch_first=True)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [18]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''



'\nAUTOGRADER CELL. DO NOT MODIFY THIS.\n'

In [19]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''



'\nAUTOGRADER CELL. DO NOT MODIFY THIS.\n'

## 3 Model Training [35 points]

### 3.1 Loss and Optimizer [5 points]

In [20]:
"""
TODO: Specify Binary Cross Entropy as the loss function (`nn.BCELoss`) and assign it to `criterion`.
      Spcify Adam as the optimizer (`torch.optim.Adam`)  with learning rate 0.001 and assign it to `optimizer`.
"""
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(naive_rnn.parameters(), lr=0.001)


# your code here
#raise NotImplementedError

In [21]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''



'\nAUTOGRADER CELL. DO NOT MODIFY THIS.\n'

### 3.2 Evaluate [10 points]

Then, let us implement the `eval_model()` function first.

In [22]:
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score


def eval_model(model, val_loader):
    
    """
    TODO: evaluate the model.
    
    Arguments:
        model: the RNN model
        val_loader: validation dataloader
        
    Outputs:
        precision: overall precision score
        recall: overall recall score
        f1: overall f1 score
        roc_auc: overall roc_auc score
        
    Note that please pass all four arguments to the model so that we can use this function for both 
    models. (Use `model(x, masks, rev_x, rev_masks)`.)
        
    HINT: checkout https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics
    """
    
    model.eval()
    y_pred = torch.LongTensor()
    y_score = torch.Tensor()
    y_true = torch.LongTensor()
    model.eval()
    for x, masks, rev_x, rev_masks, y in val_loader:
        y_hat = model(x, masks, rev_x, rev_masks)
        y_score = torch.cat((y_score,  y_hat.detach().to('cpu')), dim=0)
        y_hat = (y_hat > 0.5).int()
        y_pred = torch.cat((y_pred,  y_hat.detach().to('cpu')), dim=0)
        y_true = torch.cat((y_true, y.detach().to('cpu')), dim=0)
    """
    TODO:
        Calculate precision, recall, f1, and roc auc scores.
        Use `average='binary'` for calculating precision, recall, and fscore.
    """
  
    # your code here
    p, r, f, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
    roc_auc = roc_auc_score(y_true, y_score)
    #raise NotImplementedError
    return p, r, f, roc_auc

In [23]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''

p, r, f, roc_auc = eval_model(naive_rnn, val_loader)
assert p.size == 1, "Precision should be a scalar."
assert r.size == 1, "Recall should be a scalar."
assert f.size == 1, "F1 should be a scalar."
assert roc_auc.size == 1, "ROC-AUC should be a scalar."



### 3.3 Training and evlauation [20 points]

Now let us implement the `train()` function. Note that `train()` should call `eval_model()` at the end of each training epoch to see the results on the validaion dataset.

In [24]:
def train(model, train_loader, val_loader, n_epochs, lr=1e-3):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.BCELoss()
    
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        
        for x, masks, rev_x, rev_masks, y in train_loader:
            # 1. zero gradients
            optimizer.zero_grad()
            
            # 2. forward
            y_hat = model(x, masks, rev_x, rev_masks)
            
            # 3. compute loss
            loss = loss_fn(y_hat, y.float())
            
            # 4. backward
            loss.backward()
            
            # 5. optimizer step
            optimizer.step()
            
            # accumulate
            train_loss += loss.item()
        
        train_loss = train_loss / len(train_loader)
        print(f'Epoch: {epoch+1} \t Training Loss: {train_loss:.6f}')
        
        # Evaluate on validation set
        p, r, f, roc_auc = eval_model(model, val_loader)
        print(f'Epoch: {epoch+1} \t Validation p: {p:.2f}, r:{r:.2f}, f: {f:.2f}, roc_auc: {roc_auc:.2f}')


In [25]:
# number of epochs to train the model
n_epochs = 5
train(naive_rnn, train_loader, val_loader, n_epochs)

Epoch: 1 	 Training Loss: 0.624361
Epoch: 1 	 Validation p: 0.71, r:0.89, f: 0.79, roc_auc: 0.84
Epoch: 2 	 Training Loss: 0.441252
Epoch: 2 	 Validation p: 0.70, r:0.86, f: 0.77, roc_auc: 0.84
Epoch: 3 	 Training Loss: 0.329138
Epoch: 3 	 Validation p: 0.74, r:0.86, f: 0.80, roc_auc: 0.85
Epoch: 4 	 Training Loss: 0.224841
Epoch: 4 	 Validation p: 0.70, r:0.86, f: 0.77, roc_auc: 0.84
Epoch: 5 	 Training Loss: 0.140503
Epoch: 5 	 Validation p: 0.78, r:0.83, f: 0.81, roc_auc: 0.86


In [26]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''
p, r, f, roc_auc = eval_model(naive_rnn, val_loader)
print(roc_auc)
assert roc_auc > 0.7, "ROC AUC is too low on the validation set (%f < 0.7)"%(roc_auc)



0.8585727154438996


In [27]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''


'\nAUTOGRADER CELL. DO NOT MODIFY THIS.\n'

Perfect 👍 Let’s actually **run through the example step by step** with the simple 3-patient dataset so you can see how the `NaiveRNN` processes it.

---

### 1. Patients and their visits

We’ll use codes `{0:PAD, 1:Hypertension, 2:Diabetes, 3:CHF, 4:CAD, 5:CKD}`

* **Patient 1** → Visit 1: \[HTN, Diabetes], Visit 2: \[CHF]
* **Patient 2** → Visit 1: \[CAD], Visit 2: \[HTN, CHF, CKD]
* **Patient 3** → Visit 1: \[Diabetes], Visit 2: \[HTN], Visit 3: \[CHF, CAD]

---

### 2. Padding into tensors

We need to fit them into tensors `(B, V, C)` = `(batch, visits, codes_per_visit)`

| Patient | Visits (after padding)                |
| ------- | ------------------------------------- |
| P1      | \[\[1, 2, 0], \[3, 0, 0], \[0, 0, 0]] |
| P2      | \[\[4, 0, 0], \[1, 3, 5], \[0, 0, 0]] |
| P3      | \[\[2, 0, 0], \[1, 0, 0], \[3, 4, 0]] |

Here `0` is just padding.
We also create **masks** (1 where real code, 0 where padded).

---

### 3. What the model does

1. **Embedding layer**
   Each code gets mapped to a 128-dim vector.
   → Now each visit looks like a set of 128-dim vectors.

2. **Sum per visit**
   For example, Visit `[1, 2]` (HTN + Diabetes) becomes `embedding(1) + embedding(2)`.
   So each visit becomes a single 128-dim vector.

3. **Forward GRU**
   Reads visit embeddings in order: V1 → V2 → V3.
   Produces hidden states per visit, keeps last valid visit.

4. **Reverse GRU**
   Reads visits in reverse: V3 → V2 → V1.
   Same process, keeps last valid visit (which corresponds to the original first).

5. **Concatenate last forward + reverse states** → (256-dim).

6. **Linear + Sigmoid** → Gives probability of outcome (heart failure risk).

---

### 4. Running the model

Here’s the exact code you can run to see the predictions:

```python
with torch.no_grad():
    probs = naive_rnn(x, masks, rev_x, rev_masks)

print("Prediction probabilities per patient:")
for i, p in enumerate(probs):
    print(f"Patient {i+1}: {p.item():.4f}")
```

🔹 Example output (since weights are random, numbers will vary):

```
Patient 1: 0.4871
Patient 2: 0.5123
Patient 3: 0.4950
```

---

👉 So what you see is:

* Each patient gets a **probability score between 0 and 1**.
* With real training, this score would represent their **risk of heart failure** given their visit history.
* Right now, since the model is untrained, it outputs \~0.5 for all (randomly).

---

Would you like me to also **visualize one patient’s forward & reverse GRU hidden states** (so you can literally see how information flows visit by visit)?
